In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
csv_file_path = './airbnb_dataset/milestone3.csv'
df = pd.read_csv(csv_file_path)
pd.set_option('display.max_columns', None)
df.head()

,log_price,property_type,accommodates,bathrooms,host_response_rate,bedrooms,beds,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,city_Chicago,city_DC,city_LA,city_NYC,city_SF,cleaning_fee_True,host_has_profile_pic_t,property_type_encoded,des_sentiment_analysis,name_sentiment_analysis
0,5.010635,Apartment,-0.072621,-0.404046,0.296013,-0.312048,-0.566461,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,True,-0.245434,0.216242,-0.435248
1,5.129899,Apartment,1.783653,-0.404046,0.296013,2.034955,1.027816,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,True,True,-0.245434,0.837002,-1.788095
2,4.976734,Apartment,0.855516,-0.404046,0.296013,-0.312048,1.027816,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,True,True,-0.245434,0.808508,-0.133695
3,6.620073,House,0.391448,-0.404046,0.296013,0.861454,0.230678,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,True,0.158745,0.213627,-0.926212
4,4.744932,Apartment,-0.536689,-0.404046,0.296013,-1.485549,-0.566461,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,True,True,-0.245434,-0.101872,-0.095776


In [3]:
features = [col for col in df.columns if col != 'log_price' and df[col].dtype in ['int64', 'float64']]
X = df[features]
y = df['log_price']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
#Uncomment below if tensorflow is not installed already
#%pip install tensorflow 
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error',)

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

mse = model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

y_pred = model.predict(X_test)




Epoch 1/10

1849/1849 [==============================] - 8s 4ms/step - loss: 0.8701 - val_loss: 0.2313
Epoch 2/10
1849/1849 [==============================] - 9s 5ms/step - loss: 0.2322 - val_loss: 0.2425
Epoch 3/10
1849/1849 [==============================] - 7s 4ms/step - loss: 0.2287 - val_loss: 0.3054
Epoch 4/10
1849/1849 [==============================] - 8s 5ms/step - loss: 0.2250 - val_loss: 0.2228
Epoch 5/10
1849/1849 [==============================] - 6s 3ms/step - loss: 0.2229 - val_loss: 0.2294
Epoch 6/10
1849/1849 [==============================] - 8s 5ms/step - loss: 0.2218 - val_loss: 0.2235
Epoch 7/10
1849/1849 [==============================] - 7s 4ms/step - loss: 0.2204 - val_loss: 0.2273
Epoch 8/10
1849/1849 [==============================] - 8s 4ms/step - loss: 0.2201 - val_loss: 0.2180
Epoch 9/10
1849/1849 [==============================] - 7s 4ms/step - loss: 0.2177 - val_loss: 0.2237
Epoch 10/10
463/463 [==============================] - 1s 3ms/step - loss: 0.2

Train With XGB

In [19]:
#Uncomment if first time run
#%pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

#hyperparam may need here
param = {
    'max_depth': 5,  
    'eta': 0.3,  
    'objective': 'reg:squarederror',  
    'eval_metric': 'rmse'  
}


In [20]:
num_round = 100 
model = xgb.train(param, dtrain, num_round, evals=[(dtrain, 'train')], early_stopping_rounds=10)
y_pred = model.predict(dtrain)

mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print("Train RMSE: %f" % (rmse))
print("Train MSE: %f" % (mse))

[0]	train-rmse:0.60594
[1]	train-rmse:0.54257
[2]	train-rmse:0.50625
[3]	train-rmse:0.48591
[4]	train-rmse:0.47467
[5]	train-rmse:0.46763
[6]	train-rmse:0.46376
[7]	train-rmse:0.46135
[8]	train-rmse:0.45938
[9]	train-rmse:0.45781
[10]	train-rmse:0.45692
[11]	train-rmse:0.45622
[12]	train-rmse:0.45561
[13]	train-rmse:0.45478
[14]	train-rmse:0.45435
[15]	train-rmse:0.45394
[16]	train-rmse:0.45338
[17]	train-rmse:0.45285
[18]	train-rmse:0.45254
[19]	train-rmse:0.45227
[20]	train-rmse:0.45197
[21]	train-rmse:0.45170
[22]	train-rmse:0.45125
[23]	train-rmse:0.45103
[24]	train-rmse:0.45084
[25]	train-rmse:0.45047
[26]	train-rmse:0.45027
[27]	train-rmse:0.45000
[28]	train-rmse:0.44965
[29]	train-rmse:0.44929
[30]	train-rmse:0.44906
[31]	train-rmse:0.44877
[32]	train-rmse:0.44853
[33]	train-rmse:0.44842
[34]	train-rmse:0.44817
[35]	train-rmse:0.44800
[36]	train-rmse:0.44787
[37]	train-rmse:0.44768
[38]	train-rmse:0.44721
[39]	train-rmse:0.44688
[40]	train-rmse:0.44666
[41]	train-rmse:0.44630
[4

In [21]:
model = xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], early_stopping_rounds=10)

y_pred = model.predict(dtest)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Test RMSE: %f" % (rmse))
print("Test MSE: %f" % (mse))

[0]	test-rmse:0.61412
[1]	test-rmse:0.54937
[2]	test-rmse:0.51300
[3]	test-rmse:0.49218
[4]	test-rmse:0.48134
[5]	test-rmse:0.47424
[6]	test-rmse:0.47071
[7]	test-rmse:0.46860
[8]	test-rmse:0.46693
[9]	test-rmse:0.46522
[10]	test-rmse:0.46442
[11]	test-rmse:0.46393
[12]	test-rmse:0.46358
[13]	test-rmse:0.46290
[14]	test-rmse:0.46254
[15]	test-rmse:0.46245
[16]	test-rmse:0.46227
[17]	test-rmse:0.46186
[18]	test-rmse:0.46175
[19]	test-rmse:0.46171
[20]	test-rmse:0.46164
[21]	test-rmse:0.46152
[22]	test-rmse:0.46125
[23]	test-rmse:0.46113
[24]	test-rmse:0.46099
[25]	test-rmse:0.46078
[26]	test-rmse:0.46081
[27]	test-rmse:0.46091
[28]	test-rmse:0.46077
[29]	test-rmse:0.46075
[30]	test-rmse:0.46069
[31]	test-rmse:0.46053
[32]	test-rmse:0.46043
[33]	test-rmse:0.46040
[34]	test-rmse:0.46033
[35]	test-rmse:0.46032
[36]	test-rmse:0.46037
[37]	test-rmse:0.46031
[38]	test-rmse:0.45995
[39]	test-rmse:0.45986
[40]	test-rmse:0.45991
[41]	test-rmse:0.45982
[42]	test-rmse:0.45985
[43]	test-rmse:0.4597